# Load Data : 

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("BinancePipeline") \
    .getOrCreate()



ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
from pyspark.sql import functions as F

bronze_path='./data/btc_minute_data.parquet'

def transform_bronze_to_silver(bronze_path):
    """
    Nettoie et type les données Bronze → Silver
    """
    # Lire depuis Bronze
    df_bronze = spark.read.parquet(bronze_path)
    
    # 1. TYPAGE
    df_silver = df_bronze \
        .withColumn("open_time", F.from_unixtime(F.col("open_time")/1000).cast("timestamp")) \
        .withColumn("close_time", F.from_unixtime(F.col("close_time")/1000).cast("timestamp")) \
        .withColumn("open", F.col("open").cast("double")) \
        .withColumn("high", F.col("high").cast("double")) \
        .withColumn("low", F.col("low").cast("double")) \
        .withColumn("close", F.col("close").cast("double")) \
        .withColumn("volume", F.col("volume").cast("double")) \
        .withColumn("quote_asset_volume", F.col("quote_asset_volume").cast("double")) \
        .withColumn("taker_buy_base_volume", F.col("taker_buy_base_volume").cast("double")) \
        .withColumn("taker_buy_quote_volume", F.col("taker_buy_quote_volume").cast("double"))
    
    # 2. NETTOYAGE
    df_silver = df_silver \
        .dropDuplicates(["open_time"]) \
        .filter(F.col("close").isNotNull()) \
        .filter(F.col("volume") > 0) \
        .filter((F.col("low") <= F.col("close")) & (F.col("close") <= F.col("high"))) \
        .filter((F.col("low") <= F.col("open")) & (F.col("open") <= F.col("high"))) \
        .drop("ignore")  # Supprimer colonne inutile
    
    # 3. TRI TEMPOREL
    df_silver = df_silver.orderBy("open_time")
    
    # Sauvegarder en Silver
    silver_path = "data/silver/binance_btcusdt_clean"
    df_silver.write.mode("overwrite").parquet(silver_path)
    
    print(f"✅ {df_silver.count()} lignes nettoyées sauvegardées dans {silver_path}")
    
    return silver_path

# Exécution
silver_path = transform_bronze_to_silver(bronze_path)

In [ ]:
# Maintenant ça devrait fonctionner
bronze_path = '/mnt/c/Users/user/Desktop/Quant-AI-Project/ml/data/btc_minute_data.parquet'
df_bronze = spark.read.parquet(bronze_path)

df_bronze.show(5)
df_bronze.printSchema()

ConnectionRefusedError: [Errno 111] Connection refused